In [2]:
from deap import base, creator, tools
import random

In [ ]:
# Problem constants
# Length of bit string to be optimized
ONE_MAX_LENGTH = 100

In [5]:
toolbox = base.Toolbox()
# Define the alias of `random.randint(0, 1)` as `zeroOrOne`
toolbox.register("zeroOrOne", random.randint, 0, 1)
# Populate a `list` with `initRepeat`
randomList = tools.initRepeat(list, toolbox.zeroOrOne, 30)
print(randomList)

[0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1]


In [6]:
# Define the fitness strategy by extending the `base.Fitness` class
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

In [7]:
# Create the `Individual` class based on `list`
creator.create("Individual", list, fitness=creator.FitnessMax)

In [ ]:
# Create the `individualCreator` operator to fill up an `Individual` instance
toolbox.register("individualCreator", tools.initRepeat, creator.Individual, toolbox.zeroOrOne, ONE_MAX_LENGTH)

In [ ]:
# Create the population operator to generate a list of individuals
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)